In [1]:
import pandas as pd
from pathlib import Path

# 1. Read the file produced by `assign_polys.py`

In [2]:
# Path to data file
# Note that you may need to add an extra slash ("\") in front of the existing slashes to avoid errors
data_path = Path("C:\\Users\\tbergamaschi\\Projects\\rmp-spatial-join\\May_RMP_merged_20secMed (1)SpatialJoin.csv")

df_data = pd.read_csv(data_path)
df_data.head()

C:\Users\tbergamaschi\Anaconda3\envs\iwaas\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (101) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,FTS Timestamp,Latitude,Longitude,WS fDOM QSU,WS fCHLA ug/L,NO3 uM,CStar Tr,CStar c,TSG Temp,TSG Cond,...,Discrete PICO \nPE-RICH,Discrete Pico DENSITY (cells/L),Discrete PICO\nPC-RICH,Discrete Pico DENSITY (cells/L) .1,Discrete Pico TOTAL,Discrete Pico Notes from BSA,index_right,CL_ID,Shape_Leng,Shape_Area
0,05/15/2018 07:40:01,38.571040,-121.516280,16.460,6.912,2.919,0.595,5.187,18.914,118.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,05/15/2018 07:40:02,38.571040,-121.516280,16.463,6.973,2.941,0.595,5.190,18.912,118.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,05/15/2018 07:40:03,38.571035,-121.516300,16.469,7.037,2.963,0.595,5.195,18.910,118.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,05/15/2018 07:40:04,38.571030,-121.516320,16.476,7.100,2.941,0.595,5.200,18.907,118.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,05/15/2018 07:40:05,38.571020,-121.516325,16.482,7.091,2.919,0.595,5.200,18.906,118.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2. A little bit of clean up

It doesn't make sense to compute statistics for some columns in the dataset (like latitude and longitude), so lets identify the columns we don't want to include and drop them from the dataframe.

*Note: we could do this the other way - by identifying the columns we want to keep - but it is my hope that the columns we want to drop will be more or less consistant from dataset to dataset, more so than the constituent columns. So hopefully identifying the columns to drop will mean that little or no modification is needed from dataset to dataset. We will see...*

In [3]:
# First lets see what columns we have
df_data.columns.values

array(['FTS Timestamp', 'Latitude', 'Longitude', 'WS fDOM QSU',
       'WS fCHLA ug/L', 'NO3 uM', 'CStar Tr', 'CStar c', 'TSG Temp',
       'TSG Cond', 'TSG Salinity', 'EXO Temp', 'EXO Sp Cond', 'EXO pH',
       'EXO pH mV', 'EXO DO mg L', 'EXO DO sat', 'EXO Turb FNU',
       'EXO fDOM QSU', 'EXO fCHLA ug/L', 'EXO fBGAPC ug L',
       'Tline NH4 (uM)', 'FP Green Algae ug/L', 'FP Bluegreen ug/L',
       'FP Diatoms ug/L', 'FP Cryptophyta ug/L', 'FP Yellow substances',
       'FP Total conc. ug/L', 'FP Green Algae %', 'FP Bluegreen %',
       'FP Diatoms %', 'FP Cryptophyta %', 'FP Total conc. ug/L.1',
       'FP Transmission %', 'FP Temp. Sample deg. C', 'Discrete grNum',
       'Discrete Station', 'Discrete Station ID', 'Discrete Latitude',
       'Discrete Longitude', 'Discrete Date', 'Discrete Time',
       'Discrete Timestamp', 'Discrete Datum', 'Discrete TDN (mg-N/L)',
       'Discrete TDN (uM)', 'Discrete NH4 (mg-N/L)', 'Discrete NH4 (uM)',
       'Discrete NO2 (mg-N/L)', 'Discret

In [4]:
# Now lets make a list of the ones we don't want to include
# I am just copy/pasting non-constituent columns from above into this list
cols_to_drop = [
    'FTS Timestamp', 
    'Latitude', 
    'Longitude',
    'Discrete grNum',
       'Discrete Station', 'Discrete Station ID', 'Discrete Latitude',
       'Discrete Longitude', 'Discrete Date', 'Discrete Time',
       'Discrete Timestamp', 'Discrete Datum', 'Discrete TDN (mg-N/L)',
       'Discrete TDN (uM)', 'Discrete NH4 (mg-N/L)', 'Discrete NH4 (uM)',
       'Discrete NO2 (mg-N/L)', 'Discrete NO2 (uM)',
       'Discrete NO2+NO3 (mg-N/L)', 'Discrete NO2+NO3 (uM)',
       'Discrete Ortho-PO4 (mg-P/L)', 'Discrete Ortho-PO4 (uM)',
       'Discrete Chl-a (ug/L)', 'Discrete Pheyo. (ug/L)',
       'Discrete Total Chl (ug/L)', 'Discrete Chl-a 5um (ug/L)',
       'Discrete Pheyo. 5um (ug/L)', 'Discrete Phyco cyanin (µg/L)',
       'Discrete Allophyco cyanin (µg/L)',
       'Discrete Phyco erythrin (µg/L)', 'Discrete DOC (mg/L)',
       'Discrete TDN (mg/L)', 'Discrete A 254', 'Discrete A 280',
       'Discrete A 350', 'Discrete A 370', 'Discrete A 412',
       'Discrete A 440', 'Discrete A 488', 'Discrete A 510',
       'Discrete A 532', 'Discrete A 555', ' Discrete S ag 275 290',
       ' Discrete r 275 295', ' Discrete S ag 290 350',
       ' Discrete r 290 350', 'Discrete S ag 350 400',
       ' Discrete r 350 400', 'Discrete S ag 412 600',
       ' Discrete r 412 600', ' Discrete UV Slope Ratio',
       'Discrete  A  ex 260 em 450', 'Discrete  C ex 340 em 440',
       'Discrete  M ex 300 em 390', 'Discrete  D ex 390 em 510',
       'Discrete  B ex 275 em 304', 'Discrete  T ex 275 em 340',
       'Discrete  N ex 280 em 370', 'Discrete  fDOM ex 370 em 460',
       'Discrete  BDJF ex 420 em 460', 'Discrete  FI 2005',
       'Discrete  HI Ohno 2008', 'Discrete Pico SAMPLE\nALIQUOT (mL)',
       'Discrete Pico FIELDS', 'Discrete PICO \nPE-RICH',
       'Discrete Pico DENSITY (cells/L) ', 'Discrete PICO\nPC-RICH',
       'Discrete Pico DENSITY (cells/L) .1', 'Discrete Pico TOTAL',
       'Discrete Pico Notes from BSA', 'index_right',
    'Shape_Leng', 'Shape_Area'
]

In [5]:
df_data = df_data.drop(cols_to_drop, axis="columns")
df_data.columns.values

array(['WS fDOM QSU', 'WS fCHLA ug/L', 'NO3 uM', 'CStar Tr', 'CStar c',
       'TSG Temp', 'TSG Cond', 'TSG Salinity', 'EXO Temp', 'EXO Sp Cond',
       'EXO pH', 'EXO pH mV', 'EXO DO mg L', 'EXO DO sat', 'EXO Turb FNU',
       'EXO fDOM QSU', 'EXO fCHLA ug/L', 'EXO fBGAPC ug L',
       'Tline NH4 (uM)', 'FP Green Algae ug/L', 'FP Bluegreen ug/L',
       'FP Diatoms ug/L', 'FP Cryptophyta ug/L', 'FP Yellow substances',
       'FP Total conc. ug/L', 'FP Green Algae %', 'FP Bluegreen %',
       'FP Diatoms %', 'FP Cryptophyta %', 'FP Total conc. ug/L.1',
       'FP Transmission %', 'FP Temp. Sample deg. C', 'CL_ID'],
      dtype=object)

Cool now we just have columns that we are going to use in our statistics computation. Now we need to identify the column that contains the polygon id that was assigned by `assign_polys.py`.

In this case, the column is **`'CL_ID'`**

But keep in mind that this could be different in a different dataset. Look for something that looks like it means "centerline id" or "polygon id".

We can't do anything with rows that do not have a polygon id, so the next step will be to drop any rows with a missing polygon id

In [6]:
# Make a variable for the polygon id column name
# Change this to the name you identified in the list above
poly_id_col = "CL_ID"

print(len(df_data))
df_data = df_data.dropna(subset=[poly_id_col])
print(len(df_data))

96219
7158


We can see that the length updated if there were any rows missing a polygon id. Now we are ready to compute statistics.

# 3. Compute statistics

In [7]:
# First group the data by poly_id_col
df_grouped = df_data.groupby(poly_id_col)

In [8]:
# Now lets define the statistics we want to compute in a list that we can pass to the pandas aggregation function
# For more information on what can go into this list check out: 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html
stats = [
    "min",
    "max",
    "mean",
    "median"
]

# Compute the statistics defined above for each polygon
df_stats = df_grouped.agg(stats)

In [9]:
df_stats

WS fDOM QSU                               WS fCHLA ug/L            \
              min     max       mean     median           min       max   
CL_ID                                                                     
202.0   21.688000  29.430  24.453300  22.288500         2.574  3.763000   
203.0   29.447000  29.492  29.479818  29.492000         2.574  2.577000   
204.0   28.238263  29.492  29.039260  29.466000         2.566  2.745883   
205.0   27.987686  29.462  28.833307  29.443000         2.521  2.738661   
206.0   27.844259  29.431  28.632156  28.649771         2.485  2.765952   
...           ...     ...        ...        ...           ...       ...   
524.0   41.930000  42.321  42.127083  42.131500         2.766  2.774000   
525.0   41.744000  41.906  41.852083  41.851500         2.771  2.807000   
526.0   41.186000  41.690  41.452250  41.428000         2.808  2.826000   
527.0   40.743000  41.177  41.008727  41.057000         2.736  2.817000   
528.0   40.264000  40.632  40.450000  40.459500         2.732  2.737000   

                          NO3 uM          ... FP Total conc. ug/L.1          \
           mean   median     min     max  ...                  mean  median   
CL_ID                                     ...                                 
202.0  3.341133  3.64800  13.306  13.743  ...              3.729267  4.2390   
203.0  2.575455  2.57600  13.306  13.448  ...              2.489091  2.4900   
204.0  2.645867  2.57500  13.301  15.388  ...              2.410286  2.4670   
205.0  2.631727  2.56300  13.275  15.472  ...              2.422750  2.5325   
206.0  2.618024  2.62091  13.751  15.867  ...              2.387318  2.3710   
...         ...      ...     ...     ...  ...                   ...     ...   
524.0  2.770917  2.77100   3.231   3.279  ...              1.916917  1.9150   
525.0  2.785500  2.78300   3.001   3.244  ...              1.896333  1.8925   
526.0  2.815583  2.81700   2.885   3.057  ...              1.921667  1.9275   
527.0  2.758364  2.74500   2.787   2.886  ...              1.947727  1.9450   
528.0  2.735667  2.73600   2.818   2.880  ...              1.930917  1.9230   

      FP Transmission %                             FP Temp. Sample deg. C  \
                    min     max       mean   median                    min   
CL_ID                                                                        
202.0            94.855  98.785  97.617167  98.7050                 18.750   
203.0            95.335  95.635  95.506909  95.5100                 18.770   
204.0            93.155  96.070  94.756000  95.7500                 18.590   
205.0            93.475  96.210  95.098958  96.1050                 18.600   
206.0            94.140  96.145  95.072818  95.0250                 18.630   
...                 ...     ...        ...      ...                    ...   
524.0            76.340  76.660  76.555000  76.5850                 19.235   
525.0            76.605  76.725  76.699583  76.7250                 19.150   
526.0            75.975  76.570  76.360083  76.4100                 19.065   
527.0            75.290  75.765  75.410909  75.2900                 19.000   
528.0            75.290  76.145  75.695417  75.6875                 18.950   

                                   
          max       mean   median  
CL_ID                              
202.0  19.960  19.590800  19.9400  
203.0  18.790  18.779636  18.7800  
204.0  18.800  18.711905  18.7900  
205.0  18.800  18.714583  18.7900  
206.0  18.790  18.718455  18.7275  
...       ...        ...      ...  
524.0  19.320  19.275417  19.2700  
525.0  19.225  19.194167  19.2025  
526.0  19.143  19.098333  19.0975  
527.0  19.060  19.021818  19.0100  
528.0  18.990  18.965417  18.9675  

[237 rows x 128 columns]

# 4. Save the result

In [10]:
# Output filepath
# Make this whatever you'd like, I'm just appending "_stats" to the original filename
out_path = "May_RMP_merged_20secMed (1)SpatialJoin_stats.csv"

# Flatten the hierarchical columns
df_stats.columns = [' '.join(col).strip() for col in df_stats.columns.values]

# Write the csv
df_stats.to_csv(out_path)

And thats a wrap!